In [1]:
# ================================
# 1. Install Dependencies
# ================================
!pip install opencv-python-headless  dlib tqdm matplotlib torch torchvision
!pip install mediapipe
!pip install joblib==1.3.2 lz4==4.3.2 mtcnn==0.1.1
# ================================
# 2. Download Landmark Model for Dlib
# ================================
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
# !bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      running bdist_wheel
      running build
      running build_ext
      
      
                         CMake is not installed on your system!
      
          Or it is possible some broken copy of cmake is installed on your system.
          It is unfortunately very common for python package managers to include
          broken copies of cmake.  So if the error above this refers to some file
          path to a cmake file inside a python or anaconda or miniconda path then you
          should delete that broken copy of cmake from your computer.
      
          Instead, please get an official copy of cmake from one of these known good
          sources of an official cmake:
              - cmake.org (this is how windows users should get cmake)
              - apt install cmake (for Ubuntu or Debian based systems)
             


  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached dlib-20.0.0.tar.gz (3.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached torch-2.9.0-cp312-cp312-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.24.0-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.meta


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  error: subprocess-exited-with-error
  
  × Building wheel for lz4 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [30 lines of output]
      C:\Users\91989\AppData\Local\Temp\pip-build-env-b22zactj\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\91989\AppData\Local\Temp\pip-build-env-b22zactj\overlay\Lib\site-packages\setuptools_scm\_integration\version_inference.py:51: UserWarning: version of lz4 already set
        warnings.warn(self.message)
      C:\Users\91989\AppData\Local\Temp\pip-build-env-b22zactj\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              Lic


  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached lz4-4.3.2.tar.gz (170 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached mtcnn-0.1.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached keras-3.12.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached rich-14.2.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.1.0-py3-none-any.whl.metadata (322 bytes)
  Using cached h5py-3.15.1-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
  Using cached optree-0.17.0-cp312-cp312-win_amd64.whl.metadata (34 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached typing

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install -q datasets librosa soundfile scikit-learn joblib tqdm


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# ================================================================
# ✅ FULL DEEPFAKE AUDIO DETECTION PIPELINE
# Download → Preprocess → Train → Save Model → Test File → Delete Dataset
# ================================================================

!pip install -q datasets librosa soundfile scikit-learn joblib tqdm

import os, shutil, librosa, numpy as np, joblib
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.colab import files

# ---------------------------
# ✅ Step 1: Download dataset
# ---------------------------
dataset_name = "Hemg/Deepfake-Audio-Dataset"
dataset_cache = "/root/.cache/huggingface/datasets"

print("📥 Downloading dataset...")
dataset = load_dataset(dataset_name)

# ---------------------------
# ✅ Step 2: Extract MFCC Features
# ---------------------------
def extract_mfcc(path):
    audio, sr = librosa.load(path, sr=16000)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=30)
    return np.mean(mfcc.T, axis=0)

X, y = [], []

print("🎧 Extracting MFCC features...")
for sample in dataset['train']:
    X.append(extract_mfcc(sample["audio"]["path"]))
    y.append(1 if sample["label"]=="fake" else 0)

X, y = np.array(X), np.array(y)

# ---------------------------
# ✅ Step 3: Train/Test Split
# ---------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ---------------------------
# ✅ Step 4: Train Model
# ---------------------------
print("🤖 Training model...")
model = RandomForestClassifier(n_estimators=300)
model.fit(X_train, y_train)

# ✅ Evaluate
pred = model.predict(X_test)
print(f"✅ Accuracy: {accuracy_score(y_test, pred)*100:.2f}%")

# ---------------------------
# ✅ Step 5: Save Model
# ---------------------------
joblib.dump(model, "deepfake_detector.pkl")
print("💾 Saved model as deepfake_detector.pkl")

# ---------------------------
# ✅ Step 6: User Upload Test
# ---------------------------
print("\n📤 Upload a real/fake audio to test (.wav file)")
uploaded = files.upload()

for fn in uploaded.keys():
    print(f"🔍 Testing file: {fn}")
    
    feat = extract_mfcc(fn).reshape(1, -1)
    result = model.predict(feat)[0]
    print("🎤 Audio is:", "🔥 FAKE" if result == 1 else "✅ REAL")

# ---------------------------
# ❌ Step 7: Delete dataset
# ---------------------------
if os.path.exists(dataset_cache):
    print("🗑️ Removing dataset from system...")
    shutil.rmtree(dataset_cache)

print("\n🎉 Completed everything — dataset cleaned, model saved 🚀")


c:\Users\91989\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'google.colab'